# Initialize notebook

In [ ]:
import wandb

In [ ]:
import lpips
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
import torchmetrics
import torchvision
from torch.utils.data import Dataset, DataLoader, Subset

In [ ]:
import sys
sys.path.append("/home/e/e0425222/CS4243-project")
from utils.dataset_utils.AnimalDataset import AnimalDataset
from utils.train_utils.train_utils import visualize_results, sample_batch, summary
from utils.train_utils.model_utils import Conv2dBlock, GatedConv2dBlock, GatedUpConv2dBlock

In [32]:
train_dataset = AnimalDataset(index_file_path = "/home/e/e0425222/CS4243-project/dataset/frogs_train.txt",
    root_dir_path = "/home/e/e0425222/CS4243-project/dataset/frog_images",
    local_dir_path = "/home/e/e0425222/CS4243-project/dataset/preprocessed_64",
    file_prefix = "frogs_",
    image_dimension = 64,
    concat_mask = True,
    random_noise = False,
    require_init = False,
    drops = [])

valid_dataset = AnimalDataset(index_file_path = "/home/e/e0425222/CS4243-project/dataset/frogs_val.txt",
    root_dir_path = "/home/e/e0425222/CS4243-project/dataset/frog_images",
    local_dir_path = "/home/e/e0425222/CS4243-project/dataset/preprocessed_64",
    file_prefix = "frogs_",
    image_dimension = 64,
    concat_mask = True,
    random_noise = False,
    require_init = False,
    drops = [])

test_dataset = AnimalDataset(index_file_path = "/home/e/e0425222/CS4243-project/dataset/frogs_test.txt",
    root_dir_path = "/home/e/e0425222/CS4243-project/dataset/frog_images",
    local_dir_path = "/home/e/e0425222/CS4243-project/dataset/preprocessed_64",
    file_prefix = "frogs_",
    image_dimension = 64,
    concat_mask = True,
    random_noise = False,
    require_init = False,
    drops = [])


In [33]:
# # sanity check 
# sample_batch(train_dataset, sample_size = 6)
# sample_batch(valid_dataset, sample_size = 6)
# sample_batch(test_dataset, sample_size = 6)


# Introduction
This notebook aims to explore how contrastive learning can improve GAN performance for image inpainting. References are: 
1. Siamese Neural Networks for One-shot Image Recognition
2. SiGAN: Siamese Generative Adversarial Network for Identity-Preserving Face Hallucination
3. Free-Form Image Inpainting via Contrastive Attention Network

# Model experiment


In [34]:
class Generator(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim, activation):
        super(Generator, self).__init__()
        
        # same -> downsample -> same -> downsample
        self.conv0 = GatedConv2dBlock(input_dim, hidden_dim, kernel_size = 3, stride = 1, padding = 1, dilation = 1, activation = activation)
        self.conv1 = GatedConv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 2, padding = 1, dilation = 1, activation = activation)
        self.conv2 = GatedConv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 1, padding = 1, dilation = 1, activation = activation)
        self.conv3 = GatedConv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 2, padding = 1, dilation = 1, activation = activation)

        # 2 x same conv
        self.conv4 = GatedConv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 1, padding = 1, dilation = 1, activation = activation)
        self.conv5 = GatedConv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 1, padding = 1, dilation = 1, activation = activation)

        # 4 x dilated conv
        self.conv6 = GatedConv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 1, padding = 2, dilation = 2, activation = activation)
        self.conv7 = GatedConv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 1, padding = 4, dilation = 4, activation = activation)
        self.conv8 = GatedConv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 1, padding = 8, dilation = 8, activation = activation)
        self.conv9 = GatedConv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 1, padding = 16, dilation = 16, activation = activation)

        # 2 x same conv
        self.conv10 = GatedConv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 1, padding = 1, dilation = 1, activation = activation)
        self.conv11 = GatedConv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 1, padding = 1, dilation = 1, activation = activation)

        # upsample -> same -> upsample -> same
        self.conv12 = GatedUpConv2dBlock(hidden_dim, hidden_dim, scale_factor = (2,2), kernel_size = 3, stride = 1, padding = 1, dilation = 1, mode = 'nearest')
        self.conv13 = GatedConv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 1, padding = 1, dilation = 1, activation = activation)
        self.conv14 = GatedUpConv2dBlock(hidden_dim, hidden_dim, scale_factor = (2,2), kernel_size = 3, stride = 1, padding = 1, dilation = 1, mode = 'nearest')
        self.conv15 = GatedConv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 1, padding = 1, dilation = 1, activation = activation)

        # final
        self.final = nn.Conv2d(hidden_dim, output_dim, kernel_size = 3, stride = 1, padding = 'same')

        # for contrastive learning, 1x1 conv to compress feature map into 1 channel
        self.conv_feature = nn.Conv2d(hidden_dim, 1, kernel_size = 1, stride = 1, padding = 'same')

    def forward(self, input_tensor):

        x = self.conv0(input_tensor)
        
        # downsample
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        # middle layers
        x = self.conv4(x)
        x = self.conv5(x)

        # dilated conv with residual skips
        x = self.conv6(x) + x
        x = self.conv7(x) + x
        x = self.conv8(x) + x
        x = self.conv9(x) + x

        # extract for contrastive loss
        x_feature = self.conv_feature(x)

        # middle layers
        x = self.conv10(x)
        x = self.conv11(x)

        # upsample
        x = self.conv12(x)
        x = self.conv13(x)
        x = self.conv14(x)
        x = self.conv15(x)
        
        # final
        x = self.final(x)

        return x, x_feature


class Discriminator(nn.Module):

    def __init__(self, input_dim, hidden_dim, input_shape, activation):

        super(Discriminator, self).__init__()

        # 5 layers down
        self.conv0 = Conv2dBlock(input_dim, hidden_dim, kernel_size = 3, stride = 2, padding = 1, dilation = 1, activation = activation)
        self.conv1 = Conv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 2, padding = 1, dilation = 1, activation = activation)
        self.conv2 = Conv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 2, padding = 1, dilation = 1, activation = activation)
        self.conv3 = Conv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 2, padding = 1, dilation = 1, activation = activation)
        self.conv4 = Conv2dBlock(hidden_dim, hidden_dim, kernel_size = 3, stride = 2, padding = 1, dilation = 1, activation = activation)

        # linear to predict classes
        LATENT_H = input_shape//(2**5)
        self.linear = nn.Linear(LATENT_H**2 * hidden_dim, 1)


    def forward(self, input_tensor):
        
        x = self.conv0(input_tensor)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)

        # scores
        x = torch.flatten(x, start_dim = 1)
        x = self.linear(x)

        return x


# Training configuration

In [35]:
MODEL_PARAMS = {
    "description" : """
        Base structure same as the GLCIC model. Gated convolutions replace all convolutions in the generator, 
        and a 1x1 convolution is used to flatten the output of the 4th dilated convolution layer. 
        This is the latent vector used for contrastive learning.""",
    "hidden_dim" : 64,
    "activation" : nn.Mish,
}

SAVE_PATHS = {
    "generator1" : "/home/e/e0425222/CS4243-project/active_experiments/ContrastiveGAN/generator1/generator1",
    "generator2" : "/home/e/e0425222/CS4243-project/active_experiments/ContrastiveGAN/generator2/generator2",
    "discriminator" : "/home/e/e0425222/CS4243-project/active_experiments/ContrastiveGAN/discriminator/discriminator"
}

TRAINING_PARAMS = {
    "num_epochs" : 20,
    "batch_size" : 128, 
    "learning_rate": 1e-4,
    "alpha": 0.5,
    "beta" : 0.01,
    "ADAM_betas" : (0.5, 0.999),
    "schedule_every" : 1,
    "sample_size" : 16,
    "log_every" : 10,
    "save_paths" : SAVE_PATHS,
}

LOGGING_CONFIG = {
    "model_params" : MODEL_PARAMS,
}
LOGGING_CONFIG.update(TRAINING_PARAMS)

# Experiment intialization

In [36]:
# 1. initialize model
generator1 = Generator(input_dim = 4, hidden_dim = MODEL_PARAMS["hidden_dim"], output_dim = 3, activation = MODEL_PARAMS["activation"])
generator2 = Generator(input_dim = 4, hidden_dim = MODEL_PARAMS["hidden_dim"], output_dim = 3, activation = MODEL_PARAMS["activation"])
discriminator = Discriminator(input_dim = 3, hidden_dim = MODEL_PARAMS["hidden_dim"], input_shape = 64, activation = MODEL_PARAMS["activation"])

summary(generator1)
summary(generator2)
summary(discriminator)

# 2. device
parallel = True
device = 'cuda:0'  
devices = [0,1,2,3]

if not parallel:
    generator1 = generator1.to(device)
    generator2 = generator2.to(device)
    discriminator = discriminator.to(device)
else:
    generator1 = nn.DataParallel(generator1, device_ids = devices)
    generator2 = nn.DataParallel(generator2, device_ids = devices)
    discriminator = nn.DataParallel(discriminator, device_ids = devices)

# 3. initialize loss functions
recon_loss_function = lambda img, gt, mask : nn.functional.mse_loss(img * (1-mask), gt * (1-mask), reduction = 'sum')/(1-mask).sum()
contrastive_loss_function_same = lambda x1, x2 : nn.functional.mse_loss(x1, x2)
discriminator_loss_function = nn.functional.binary_cross_entropy_with_logits

# 4. initialize metrics
VGG_LPIPS = lpips.LPIPS(net = 'vgg').to(device)
METRICS = {
    "Peak SnR (Whole)" : lambda img, gt, mask : torchmetrics.functional.peak_signal_noise_ratio(img * (1-mask) + gt * mask, gt),
    "L2 loss (Whole)" : lambda img, gt, mask : nn.functional.mse_loss(img * (1-mask) + gt * mask, gt),
    "L2 loss (Mask)" : lambda img, gt, mask : nn.functional.mse_loss(img * (1-mask), gt * (1-mask), reduction = 'sum')/(1-mask).sum(),
    "L1 loss (Whole)" : lambda img, gt, mask : nn.functional.l1_loss(img * (1-mask) + gt * mask, gt),
    "L1 loss (Mask)" : lambda img, gt, mask : nn.functional.l1_loss(img * (1-mask), gt * (1-mask), reduction = 'sum')/(1-mask).sum(),
    "LPIPS (Whole)" : (lambda img, gt, mask : VGG_LPIPS(img * (1-mask) + gt * mask, gt).mean()),
}


# 5. initialize optimizers
generator1_optimizer = torch.optim.Adam(generator1.parameters(), lr = TRAINING_PARAMS["learning_rate"], betas = TRAINING_PARAMS["ADAM_betas"])
generator1_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(generator1_optimizer, mode = 'min', factor = 0.5, patience = 1, threshold = 1e-6)

generator2_optimizer = torch.optim.Adam(generator2.parameters(), lr = TRAINING_PARAMS["learning_rate"], betas = TRAINING_PARAMS["ADAM_betas"])
generator2_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(generator2_optimizer, mode = 'min', factor = 0.5, patience = 1, threshold = 1e-6)

discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr = TRAINING_PARAMS["learning_rate"], betas = TRAINING_PARAMS["ADAM_betas"])
discriminator_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(discriminator_optimizer, mode = 'min', factor = 0.5, patience = 1, threshold = 1e-6)

# 6. wrap into training dictionary
TRAINING_PARAMS["generator1_model"] = generator1
TRAINING_PARAMS["generator1_optimizer"] = generator1_optimizer
TRAINING_PARAMS["generator1_scheduler"] = generator1_scheduler
TRAINING_PARAMS["generator2_model"] = generator2
TRAINING_PARAMS["generator2_optimizer"] = generator2_optimizer
TRAINING_PARAMS["generator2_scheduler"] = generator2_scheduler
TRAINING_PARAMS["discriminator_model"] = discriminator
TRAINING_PARAMS["discriminator_optimizer"] = discriminator_optimizer
TRAINING_PARAMS["discriminator_scheduler"] = discriminator_scheduler
TRAINING_PARAMS["discriminator_loss_function"] = discriminator_loss_function
TRAINING_PARAMS["contrastive_loss_function"] = contrastive_loss_function_same
TRAINING_PARAMS["recon_loss_function"] = recon_loss_function



model has 1.11642 million parameters
model has 1.11642 million parameters
model has 0.150401 million parameters
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /home/e/e0425222/miniconda3/envs/env/lib/python3.7/site-packages/lpips/weights/v0.1/vgg.pth


# Intialize logging

In [37]:
wandb.init(project="ContrastiveGAN", entity="cs4243_project")
wandb.config = LOGGING_CONFIG

wandb.watch(
    (generator1, generator2, discriminator),
    criterion = None,
    log = 'all',
    log_freq = 1,
    idx = 0, 
    log_graph = False
)


L1 loss (Mask),█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
L1 loss (Whole),█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
L2 loss (Mask),█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
L2 loss (Whole),█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
LPIPS (Whole),█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Peak SnR (Whole),▁▆▆▇▇▇▇▇▇▇▇▇▇███████████████████████████
eval_L1 loss (Mask),██▆▆▅▅▄▄▇▇▃▄▆▅▄▅▃▃▅▅▃▄▂▃▂▂▂▁▃▂▂▃▃▂▁▁▁▁▁▁
eval_L1 loss (Whole),██▇▇▅▄▄▄▇▇▂▃▆▅▅▅▃▃▆▅▃▄▂▃▂▂▁▁▃▂▂▃▂▂▂▁▁▁▁▁
eval_L2 loss (Mask),██▆▆▅▄▄▄▆▇▂▃▅▅▅▅▃▃▄▄▃▄▂▃▂▂▂▁▃▂▂▂▃▂▁▁▁▁▁▁
eval_L2 loss (Whole),██▆▆▅▄▄▄▆▇▂▃▅▅▅▅▃▃▅▄▃▄▂▃▂▂▂▁▃▂▂▂▃▂▁▁▁▁▁▁
eval_LPIPS (Whole),▇▇██▆▆▇▆██▆▆▅▆▇▆▅▅▇▆▄▄▄▄▃▂▁▂▂▂▁▂▂▂▂▂▂▁▂▂


[]

# Train

## Train functions

In [38]:

def train_epoch(device, train_dataloader, training_params : dict, metrics : dict, log_wandb = True):
    
    # ===== INITIALIZE =====
    # constants
    RECONSTRUCTION_LOSS = training_params["recon_loss_function"]
    DISCRIMINATOR_LOSS_FUNCTION = training_params["discriminator_loss_function"]
    CONTRASTIVE_LOSS_FUNCTION = training_params["contrastive_loss_function"]

    GENERATOR1_OPTIMIZER = training_params["generator1_optimizer"]
    GENERATOR2_OPTIMIZER = training_params["generator2_optimizer"]
    DISCRIMINATOR_OPTIMIZER = training_params["discriminator_optimizer"]

    BATCH_EVALUATE_EVERY = 5
    LOG_EVERY = training_params["log_every"]
    SAMPLE_SIZE = training_params["sample_size"]
    BATCH_SIZE = training_params["batch_size"]
    ALPHA = training_params["alpha"]
    BETA = training_params["beta"]

    # models
    generator1 = training_params["generator1_model"].to(device).train()
    generator2 = training_params["generator2_model"].to(device).train()
    discriminator = training_params["discriminator_model"].to(device).train()

    # epoch metrics
    running_results = {list(metrics.keys())[i] : 0.0 for i in range(len(metrics)) } 
    running_results["loss_discriminator"] = 0.0
    running_results["loss_generator1"] = 0.0
    running_results["loss_generator1_r"] = 0.0
    running_results["loss_generator1_adv"] = 0.0
    running_results["loss_generator2"] = 0.0
    running_results["loss_generator2_r"] = 0.0
    running_results["loss_generator2_adv"] = 0.0
    running_results["loss_contrastive"] = 0.0

    # ===== TRAIN EPOCH =====
    num_batches = 0
    for _, batch in enumerate(train_dataloader, 1):

            # ===== INITIALIZE =====
            num_batches += 1

            # input and ground truth
            input_batched = batch["image"]
            ground_truth_batched = batch["reconstructed"]
            mask_batched = batch["mask"]

            # sanity check
            assert input_batched.shape[0] == ground_truth_batched.shape[0]

            # move tensors to device
            input_batched = input_batched.to(device)
            ground_truth_batched = ground_truth_batched.to(device)
            mask_batched = mask_batched.to(device)

            # set the gradients to zeros
            GENERATOR1_OPTIMIZER.zero_grad()
            GENERATOR2_OPTIMIZER.zero_grad()
            DISCRIMINATOR_OPTIMIZER.zero_grad()

            # reshape to channel first
            input_batched = input_batched.permute(0, 3, 1, 2)
            ground_truth_batched = ground_truth_batched.permute(0, 3, 1, 2)
            mask_batched = mask_batched.permute(0, 3, 1, 2)

            # ===== FORWARD PASS =====

            # 1. train discriminator
            
            # 1.1 generate images
            input_batched.requires_grad_()
            output1_batched, _ = generator1(input_batched)
            shuffled_indices = torch.randperm(n = BATCH_SIZE)
            output2_batched, _ = generator2(input_batched[shuffled_indices]) # random permutation

            # 1.2 splice with ground truth
            spliced1_batched = ((1-mask_batched) * output1_batched) + (mask_batched * ground_truth_batched) 
            spliced2_batched = ((1-mask_batched[shuffled_indices]) * output2_batched) + (mask_batched[shuffled_indices] * ground_truth_batched[shuffled_indices]) 

            # 1.3 feed into discriminator
            label_real = torch.ones(BATCH_SIZE * 2, 1).to(device)
            label_fake = torch.zeros(BATCH_SIZE * 2, 1).to(device)

            pred_real = discriminator(torch.cat([ground_truth_batched, ground_truth_batched[shuffled_indices]], dim = 0))
            loss_real = DISCRIMINATOR_LOSS_FUNCTION(pred_real, label_real)

            pred_fake = discriminator(torch.cat([spliced1_batched, spliced2_batched[shuffled_indices]], dim = 0))
            loss_fake = DISCRIMINATOR_LOSS_FUNCTION(pred_fake, label_fake)
            loss_d = ALPHA * (loss_real + loss_fake)
            loss_d.backward()
            DISCRIMINATOR_OPTIMIZER.step()

            # 2. train generator 1 (reconstruction, adverserial, contrastive)
            # 2.1 forward pass by generator to produce images, splice them
            output1_batched, _ = generator1(input_batched)
            spliced1_batched = ((1-mask_batched) * output1_batched) + (mask_batched * ground_truth_batched) 
            
            # 2.2 reconstruction loss
            loss_g1_reconstruction = RECONSTRUCTION_LOSS(output1_batched, ground_truth_batched, mask_batched)

            # 2.3 adverserial loss
            label_real = torch.ones(BATCH_SIZE, 1).to(device)
            pred1_adverserial = discriminator(spliced1_batched)
            loss_g1_adverserial = DISCRIMINATOR_LOSS_FUNCTION(pred1_adverserial, label_real) # want it to classify all as real

            # 2.4 contrastive loss
            shuffled_indices = torch.randperm(n = BATCH_SIZE)
            copied_input_batched = torch.cat([ground_truth_batched * mask_batched[shuffled_indices], mask_batched[shuffled_indices]], dim = 1)
            _, x1 = generator1(input_batched)
            _, x2 = generator2(copied_input_batched) # same image, different masks
            loss_contrastive1 = CONTRASTIVE_LOSS_FUNCTION(x1, x2)

            # 2.5 backprop
            loss_g1 = loss_g1_reconstruction + ALPHA * loss_g1_adverserial + BETA * loss_contrastive1
            loss_g1.backward()
            GENERATOR1_OPTIMIZER.step()

            # 3.  train generator 2 (reconstruction, adverserial, contrastive)
            # 3.1 forward pass by generator to produce images, reconstruction loss
            output2_batched, _ = generator2(input_batched)
            spliced2_batched = ((1-mask_batched) * output2_batched) + (mask_batched * ground_truth_batched) 
            loss_g2_reconstruction = RECONSTRUCTION_LOSS(output2_batched, ground_truth_batched, mask_batched)

            # 3.2 adverserial loss
            label_real = torch.ones(BATCH_SIZE, 1).to(device)
            pred2_adverserial = discriminator(spliced2_batched)
            loss_g2_adverserial = DISCRIMINATOR_LOSS_FUNCTION(pred2_adverserial, label_real) # want it to classify all as real

            # 3.3 contrastive loss
            shuffled_indices = torch.randperm(n = BATCH_SIZE)
            copied_input_batched = torch.cat([ground_truth_batched * mask_batched[shuffled_indices], mask_batched[shuffled_indices]], dim = 1)
            _, x1 = generator1(input_batched)
            _, x2 = generator2(copied_input_batched) # same image, different masks
            loss_contrastive2 = CONTRASTIVE_LOSS_FUNCTION(x1, x2)

            # 3.4 backprop
            loss_g2 = loss_g2_reconstruction + ALPHA * loss_g2_adverserial + BETA * loss_contrastive2
            loss_g2.backward()
            GENERATOR2_OPTIMIZER.step()


            # ===== COMPUTE STATISTICS, USING TORCH METRICS =====  
            # 1. compute losses

            running_results["loss_generator1"] += loss_g1.detach().item()
            running_results["loss_generator1_r"] += loss_g1_reconstruction.detach().item()
            running_results["loss_generator1_adv"] += loss_g1_adverserial.detach().item()
            running_results["loss_generator2"] += loss_g2.detach().item()
            running_results["loss_generator2_r"] += loss_g2_reconstruction.detach().item()
            running_results["loss_generator2_adv"] += loss_g2_adverserial.detach().item()
            running_results["loss_contrastive"] += (loss_contrastive1.detach().item() + loss_contrastive2.detach().item())/2
            running_results["loss_discriminator"] += loss_d.detach().item()

            # 2. for each key, compute, add item to results dictionary (take average of 2 generators)
            for key, func in metrics.items():
                res1 = func(output1_batched, ground_truth_batched, mask_batched).detach().item()
                res2 = func(output2_batched, ground_truth_batched, mask_batched).detach().item()
                running_results[key] += (res1 + res2)/2

            # 3. log with wandb
            if log_wandb and (num_batches % LOG_EVERY == 0):

                # generator 1
                batched_predictions = torch.cat([
                    input_batched[:SAMPLE_SIZE, 0:3,:,:], # can be 4 channels
                    ground_truth_batched[:SAMPLE_SIZE,:,:,:],  # 3 channels
                    spliced1_batched[:SAMPLE_SIZE,:,:,:]], dim = 0) 
                
                image_array = torchvision.utils.make_grid(batched_predictions, nrow = SAMPLE_SIZE, padding = 50)
                images1 = wandb.Image(
                    image_array, 
                    caption = "1st row: Damaged, 2nd row: Ground truth, 3rd row: spliced")
                
                # generator 2
                batched_predictions = torch.cat([
                    input_batched[:SAMPLE_SIZE, 0:3,:,:], # can be 4 channels
                    ground_truth_batched[:SAMPLE_SIZE,:,:,:],  # 3 channels
                    spliced2_batched[:SAMPLE_SIZE,:,:,:]], dim = 0) 
                
                image_array = torchvision.utils.make_grid(batched_predictions, nrow = SAMPLE_SIZE, padding = 50)
                images2 = wandb.Image(
                    image_array, 
                    caption = "1st row: Damaged, 2nd row: Ground truth, 3rd row: spliced")

                # log images and some metadata
                wandb.log( {
                    "generator1_train_images" : images1,
                    "generator2_train_images" : images2,
                    "lr_generator1" : GENERATOR1_OPTIMIZER.param_groups[0]['lr'],
                    "lr_generator2" : GENERATOR2_OPTIMIZER.param_groups[0]['lr'],
                    "lr_discriminator" : DISCRIMINATOR_OPTIMIZER.param_groups[0]['lr']

                })

                # log all metrics
                wandb.log(
                    {key : item/num_batches for key, item in running_results.items()}
                )
        

            # ===== HOUSEKEEPING =====
            del loss_g2
            del loss_g1
            del loss_d
            del input_batched

            # print results every some batches
            if num_batches % BATCH_EVALUATE_EVERY == 0: 

                args = ""
                for key, val in running_results.items():
                    args += key + ": " + str(running_results[key]/num_batches) + "   "
                print(f"\r{num_batches}/{len(train_dataloader)}: " + args, end = '', flush = True)

    # normalise numbers by batch
    for key, val in running_results.items():
        running_results[key] /= num_batches

    return running_results




In [39]:
def evaluate_epoch(device, validation_dataloader, training_params : dict, metrics : dict, log_wandb = True):

    # ===== INITIALIZE =====
    # models
    generator1 = training_params["generator1_model"].to(device).eval()
    generator2 = training_params["generator2_model"].to(device).eval()

    # constants
    SAMPLE_SIZE = training_params["sample_size"]

    # epoch statistics
    running_results = {list(metrics.keys())[i] : 0.0 for i in range(len(metrics)) } 

    # ===== EVALUATE EPOCH =====

    with torch.no_grad():
        batches = 0
        for index, batch in enumerate(validation_dataloader, 1):
            
            batches += 1

            # input and ground truth
            input_batched = batch["image"]
            ground_truth_batched = batch["reconstructed"]
            mask_batched = batch["mask"]

            # move tensors to device
            input_batched = input_batched.to(device)
            ground_truth_batched = ground_truth_batched.to(device)
            mask_batched = mask_batched.to(device)

            # ===== FORWARD PASS =====

            # 1. reshape to channel first
            input_batched = input_batched.permute(0, 3, 1, 2)
            ground_truth_batched = ground_truth_batched.permute(0, 3, 1, 2)
            mask_batched = mask_batched.permute(0, 3, 1, 2)

            # 2. predict    
            output1_batched, _ = generator1(input_batched)
            output2_batched, _ = generator2(input_batched)           
            spliced1_batched = ((1-mask_batched) * output1_batched) + (mask_batched * ground_truth_batched) 
            spliced2_batched = ((1-mask_batched) * output2_batched) + (mask_batched * ground_truth_batched) 

            # 3. evaluate
            for key, func in metrics.items():
                running_results[key] += (func(output1_batched, ground_truth_batched, mask_batched).detach().item() + func(output2_batched, ground_truth_batched, mask_batched).detach().item())/2

            args = ""
            for key, val in running_results.items():
                args += key + ": " + str(running_results[key]/batches) + "   "
            print(f"\r{batches}/{len(validation_dataloader)}: " + args, end = '', flush = True)

            # 4. log 
            if log_wandb:

                # generator 1
                batched_predictions = torch.cat([
                    input_batched[:SAMPLE_SIZE, 0:3,:,:], # can be 4 channels
                    ground_truth_batched[:SAMPLE_SIZE,:,:,:],  # 3 channels
                    spliced1_batched[:SAMPLE_SIZE,:,:,:]], dim = 0) 
                
                image_array = torchvision.utils.make_grid(batched_predictions, nrow = SAMPLE_SIZE, padding = 50)
                images1 = wandb.Image(
                    image_array, 
                    caption = "1st row: Damaged, 2nd row: Ground truth, 3rd row: spliced")
                
                # generator 2
                batched_predictions = torch.cat([
                    input_batched[:SAMPLE_SIZE, 0:3,:,:], # can be 4 channels
                    ground_truth_batched[:SAMPLE_SIZE,:,:,:],  # 3 channels
                    spliced2_batched[:SAMPLE_SIZE,:,:,:]], dim = 0) 
                
                image_array = torchvision.utils.make_grid(batched_predictions, nrow = SAMPLE_SIZE, padding = 50)
                images2 = wandb.Image(
                    image_array, 
                    caption = "1st row: Damaged, 2nd row: Ground truth, 3rd row: spliced")

                # log images and some metadata
                wandb.log( {
                    "generator1_val_images" : images1,
                    "generator2_val_images" : images2
                })

                # log all metrics
                wandb.log(
                    {f"eval_{key}" : item/batches for key, item in running_results.items()}
                )

    # normalise numbers by batch
    for key, val in running_results.items():
        running_results[key] /= batches

    return running_results


In [40]:
def train_evaluate(device, train_dataset, validation_dataset, training_params: dict, metrics: dict, start_epoch = 0, log_wandb = True):

    # ===== INITIALIZE =====
    # constants
    NUM_EPOCHS = training_params["num_epochs"]
    BATCH_SIZE = training_params["batch_size"]
    GENERATOR1_SCHEDULER = training_params["generator1_scheduler"]
    GENERATOR2_SCHEDULER = training_params["generator2_scheduler"]
    DISCRIMINATOR_SCHEDULER = training_params["discriminator_scheduler"]
    SAVE_PATHS = training_params["save_paths"]
    NUM_WORKERS = 2
    START_EPOCH = start_epoch

    # models for saving
    generator1 = training_params["generator1_model"].to(device).train()
    generator2 = training_params["generator2_model"].to(device).train()
    discriminator = training_params["discriminator_model"].to(device).train()

    # variable losses
    train_results = {list(metrics.keys())[i] : [] for i in range(len(metrics)) } 

    train_results["loss_discriminator"] = []
    train_results["loss_generator1"] = []
    train_results["loss_generator1_r"] = []
    train_results["loss_generator1_adv"] = []
    train_results["loss_generator2"] = []
    train_results["loss_generator2_r"] = []
    train_results["loss_generator2_adv"] = []
    train_results["loss_contrastive"] = []

    eval_results = {list(metrics.keys())[i] : [] for i in range(len(metrics)) } 

    # dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = NUM_WORKERS, drop_last = True)
    validation_dataloader = DataLoader(validation_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = NUM_WORKERS, drop_last = True)

    # ===== TRAIN =====
    for epoch in range(NUM_EPOCHS):

        start = time.time()

        # train
        print(f"\n===== Epoch: {START_EPOCH + epoch + 1} ===== ")
        num_batches = 0

        # train every epoch
        results = train_epoch(device, train_dataloader, training_params, metrics, log_wandb = log_wandb)
        for key, val in results.items():
            train_results[key].append(val)

        # evaluate every epoch
        print()
        results = evaluate_epoch(device, validation_dataloader, training_params, metrics, log_wandb = log_wandb)
        for key, val in results.items():
            eval_results[key].append(val)

        # ===== EPOCH RESULTS =====
        print(f"\nCompleted epoch {START_EPOCH + epoch + 1}! Took {(time.time() - start)/60} min")

        # ===== HOUSEKEEPING =====

        # scheduler every epoch
        if DISCRIMINATOR_SCHEDULER is not None:
            DISCRIMINATOR_SCHEDULER.step(eval_results["L2 loss (Mask)"][epoch])
        if GENERATOR1_SCHEDULER is not None:
            GENERATOR1_SCHEDULER.step(eval_results["L2 loss (Mask)"][epoch])
        if GENERATOR2_SCHEDULER is not None:
            GENERATOR2_SCHEDULER.step(eval_results["L2 loss (Mask)"][epoch])
            
        # save every epoch
        SAVE = f"{SAVE_PATHS['generator1']}_epoch{START_EPOCH + epoch + 1}.pt"
        torch.save(generator1.state_dict(), SAVE)
        SAVE = f"{SAVE_PATHS['generator2']}_epoch{START_EPOCH + epoch + 1}.pt"
        torch.save(generator2.state_dict(), SAVE)
        SAVE = f"{SAVE_PATHS['discriminator']}_epoch{START_EPOCH + epoch + 1}.pt"
        torch.save(discriminator.state_dict(), SAVE)  

        print("Saved models!")

    return train_results, eval_results

## Train

In [42]:
torch.autograd.set_detect_anomaly(True)

train_evaluate(device, train_dataset, valid_dataset, TRAINING_PARAMS, METRICS, start_epoch = 1, log_wandb = True)


===== Epoch: 2 ===== 
75/78: Peak SnR (Whole): 30.71053050994873   L2 loss (Whole): 0.0008522870379965753   L2 loss (Mask): 0.07410091881950696   L1 loss (Whole): 0.004203918441198766   L1 loss (Mask): 0.3655221309264501   LPIPS (Whole): 0.042867486774921415   loss_discriminator: 0.1389591598510742   loss_generator1: 0.15367983877658845   loss_generator1_r: 0.0771857231358687   loss_generator1_adv: 0.7015039857228597   loss_generator2: 0.1474274315436681   loss_generator2_r: 0.07101611450314521   loss_generator2_adv: 0.7009817926088969   loss_contrastive: 0.6328426249821981       
7/7: Peak SnR (Whole): 31.09635434831892   L2 loss (Whole): 0.0007771012439791645   L2 loss (Mask): 0.06753567659429141   L1 loss (Whole): 0.0039976856704535225   L1 loss (Mask): 0.3474681334836142   LPIPS (Whole): 0.04218321852385998    
Completed epoch 2! Took 5.123102037111918 min
Saved models!

===== Epoch: 3 ===== 
75/78: Peak SnR (Whole): 31.144846852620443   L2 loss (Whole): 0.0007694351855510225   L2

wandb: Network error (ReadTimeout), entering retry loop.


75/78: Peak SnR (Whole): 31.9257675298055   L2 loss (Whole): 0.0006435682492641112   L2 loss (Mask): 0.05581380488971869   L1 loss (Whole): 0.0035833495234449705   L1 loss (Mask): 0.3107676794131597   LPIPS (Whole): 0.03798622121413549   loss_discriminator: 0.1365488823254903   loss_generator1: 0.13222080320119858   loss_generator1_r: 0.057326476176579794   loss_generator1_adv: 0.7291651407877604   loss_generator2: 0.12894901216030122   loss_generator2_r: 0.05430113360285759   loss_generator2_adv: 0.726722104549408   loss_contrastive: 0.19767392347256343      
7/7: Peak SnR (Whole): 32.40780626024519   L2 loss (Whole): 0.0005743799390204783   L2 loss (Mask): 0.05006111493068082   L1 loss (Whole): 0.0033240020607731174   L1 loss (Mask): 0.2897902599402836   LPIPS (Whole): 0.035420920965926986    
Completed epoch 6! Took 5.051364807287852 min
Saved models!

===== Epoch: 7 ===== 
75/78: Peak SnR (Whole): 32.003800684611   L2 loss (Whole): 0.000632371447669963   L2 loss (Mask): 0.054662578

({'Peak SnR (Whole)': [30.735400676727295,
   31.151127619621082,
   31.459579272147938,
   31.692329492324436,
   31.941340311979637,
   32.015968934083595,
   32.15032104345468,
   32.21265817299867,
   32.22223741580279,
   32.407617079905975,
   32.43529154704167,
   32.44203859720475,
   32.28815243794368,
   32.29551962094429,
   32.43759193175878,
   32.547223115578674,
   32.40239258301564,
   32.45111541258983,
   32.36448396780552,
   32.47878331404466],
  'L2 loss (Whole)': [0.0008477815314948272,
   0.0007682936407852536,
   0.0007162326573728559,
   0.000678657080219963,
   0.0006414199552427118,
   0.0006306035218772311,
   0.0006106423933464938,
   0.000601833061059901,
   0.0006005199799964873,
   0.0005756769083419408,
   0.0005716364603125466,
   0.0005718370276810721,
   0.0005914723873362304,
   0.0005900680027060354,
   0.0005704453308284521,
   0.0005567698168213097,
   0.0005756215189019433,
   0.0005689643790575269,
   0.0005804705687273198,
   0.000565541398641

In [ ]:
# train_epoch(device, DataLoader(train_dataset, batch_size = 16), TRAINING_PARAMS, METRICS, log_wandb = True)
# evaluate_epoch(device, DataLoader(valid_dataset, batch_size = 16),, TRAINING_PARAMS, metrics, log_wandb = True)